In [4]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time          

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.수집할 자료의 키워드는 무엇입니까? : ')

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
ft_name = "C:\\Users\\Windows\\Desktop\\대학교\\4학년 여름방학\\Big_AI\\크롤링\\코드\\Chap_3\\출력 파일\\riss.txt"
fc_name = "C:\\Users\\Windows\\Desktop\\대학교\\4학년 여름방학\\Big_AI\\크롤링\\코드\\Chap_3\\출력 파일\\riss.csv"
fx_name = "C:\\Users\\Windows\\Desktop\\대학교\\4학년 여름방학\\Big_AI\\크롤링\\코드\\Chap_3\\출력 파일\\riss.xls"

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
s = Service("C:\\chromedriver.exe")
driver = webdriver.Chrome(service=s)

url = 'https://www.riss.kr/'
driver.get(url)
time.sleep(3)
driver.maximize_window()

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element(By.ID,'query')
driver.find_element(By.ID,'query').click( )
element.send_keys(query_txt)
element.send_keys("\n")

#Step 6.학위 논문 선택하기
driver.find_element(By.LINK_TEXT,'학위논문').click()
time.sleep(2)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','srchResultListW').find_all('li')
for i in content_1 :
    print(i.get_text().replace("\n",""))

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox pd').find('span','num').get_text()
print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10) # 올림
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))
print('=' *80)

#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
no = 1

for a in range(1, collect_page_cnt + 1) :
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for b in content_2 :    
        #1. 논문제목 있을 경우만
        try :
            title = b.find('div','cont').find('p','title').get_text()
        except :
            continue
        else :
            f = open(ft_name, 'w' , encoding="UTF-8")
            print('1.번호:',no)
            no2.append(no)
            f.write('\n'+'1.번호:' + str(no))

            print('2.논문제목:',title)
            title2.append(title)
            f.write('\n' + '2.논문제목:' + title)
            
            writer = b.find('span','writer').get_text()
            print('3.저자:',writer)
            writer2.append(writer)
            f.write('\n' + '3.저자:' + writer)

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            f.write('\n' + '4.소속기관:' + org + '\n')
            
            f.close( )
            
            no += 1
            print("\n")
            
            if no > collect_cnt :
                break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

    a += 1 
    b = str(a)

    try :
        driver.find_element(By.LINK_TEXT ,'%s' %b).click() 
    except :
        driver.find_element(By.LINK_TEXT, '다음 페이지로').click()
        
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2) # 결측값이 있으면 df["제목"] = title2가 작동 x
df['저자']=pd.Series(writer2)
df['소속(발행)기관']=pd.Series(org2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8" , engine='openpyxl')

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까? : 전염병
1전염병 모델의 연대기적 분석과 후속모델에 대한 고찰이상원高麗大學校 大學院2012국내석사RANK : 27772927원문보기목차검색조회음성듣기전염병은 고대 시대부터 줄곧 인간을 위협해왔으며, 기원전 430년부터 428년에 발생한 아테네지역의 전염병이 Thucydides에 의해 보고된 이후, 전 세계적으로 많은 사망자를 발생하게 한 전염병들이 다수의 생물학자들에 의해 분석되어 졌다. 전염병에 대한 연구가 2차 세계대전 이전에는 단순히 전염병을 관찰하는데 그친 반면, 2차 대전이후에는 전염병을 무기화 하고 그것을 어떻게 통제할 것인지에 초점이 맞춰지기 시작하였다. 또한 페니실린에 발견으로 이러한 질병을 통제하는 방법이 다변화되었으며, Bernoulli가 천연두의 확산에 대해 우두접종의 긍정적인 효과를 보고 한 후, Kermack-McKendrick에 의해 수학에 기초를 둔 모델링이 시작되었다. 이는 생물학적인 분석에 수학적인 사고를 접목하여, 전염병을 좀 더 체계적으로 분석하고자 하는 시도로 볼 수 있다.  Anderson-May는 좀 더 이러한 연구들을 발전시켰으며, 현재 많은 연구자들에 의해 새로운 발생하는 전염병들에 대해 연구가 이루어지고 있다. 특히, 상미분방정식과 편미분방정식을 도구로 하여 여러 가지 형태의 전염병 모델을 발전시킴으로써 전염병을 효과적으로 통제할 수 있는 연구가 활발히 진행되어 지고 있다.  그동안 많은 연구에서 전염병을 통제하는데 가장 중요한 요소는 전염병 전파에 의해 영향 받지 않는 점(Disease free equilibrium : DFE)에 대한 의 값으로서, 그 값이  보다 크거나 혹은 작은 경우에 따라 전염병의 통제 여부를 판단하여 왔으나, 최근에는 통제 재생산률(Control reproduction rate : 모델링에서 각종 통제 변수들에 의해 변동되어지는 질병이 전파되는 비율)라는 값이 전염병을 통제하는

이 중에서 몇 건을 수집하시겠습니까?: 102
102 건의 데이터를 수집하기 위해 11 페이지의 게시물을 조회합니다.
1.번호: 1
2.논문제목: 전염병 모델의 연대기적 분석과 후속모델에 대한 고찰
3.저자: 이상원
4.소속기관: 高麗大學校 大學院


1.번호: 2
2.논문제목: 전염병 위험지각이 관광행동의도에 미치는 영향
3.저자: 김혜진
4.소속기관: 세종대학교 대학원


1.번호: 3
2.논문제목: 코로나(COVID-19) 등 호흡기전염병 병원 내 감염 예방을 위한 표준개발
3.저자: 황창의
4.소속기관: 계명대학교대학원


1.번호: 4
2.논문제목: 국제보건규칙 개정에 따른 우리나라 검역전염병 지정방안
3.저자: 김윤호
4.소속기관: 경북대학교 보건대학원


1.번호: 5
2.논문제목: 전염병과 펜데믹 상황에서 메디컬 처치의 운영에 관한 연구
3.저자: 이재훈
4.소속기관: 칼빈대학교 신학대학원


1.번호: 6
2.논문제목: 전염병으로 인한 사회적 변화로서 제한적 일상 표현 연구 : COVID-19 팬데믹을 소재로 한 연구작을 중심으로
3.저자: 박연경
4.소속기관: 부산대학교


1.번호: 7
2.논문제목: 전염병에 대한 기독교 견해와 고찰
3.저자: 이성훈
4.소속기관: 칼빈대학교 신학대학원


1.번호: 8
2.논문제목: 국제법상 전염병 통제에 관한 연구
3.저자: 박진아
4.소속기관: 고려대학교 대학원


1.번호: 9
2.논문제목: 항공운송을 통해 전파되는 전염병의 확산방지를 위한 국제적 대응 연구
3.저자: 최성희
4.소속기관: 韓國外國語大學校 政治行政言論大學院


1.번호: 10
2.논문제목: 조선총독부의 급성전염병 예방 대책 변화 - 수인성 전염병을 중심으로
3.저자: 백선례
4.소속기관: 한양대학교 대학원


1.번호: 11
2.논문제목: 학령전 아동을 위한 호흡기전염병 예방 프로그램의 개발 및 효과에 관한 연구
3.저자: 김일옥
4.소속기관: 梨花女子大學校


1.번호: 12
2.논문제목: 우리나라 법정전염병 관리실태 및 예방대책에

1.번호: 92
2.논문제목: 최근 국내에서 발생된 중요법정전염병에 관한 보건통계학적 연구
3.저자: 인기영
4.소속기관: 경희대학교


1.번호: 93
2.논문제목: GIS를 이용한 전염병 발병 예측 시스템 설계 및 구현 (말라리아를 중심으로)
3.저자: 공용우
4.소속기관: 仁荷大學校 産業大學院


1.번호: 94
2.논문제목: 감염질병의 역학 : 전염병 모델과 AIDS
3.저자: 곽다니
4.소속기관: 성신여자대학교


1.번호: 95
2.논문제목: 일제하 한국에서의 전염병 발생과 대책
3.저자: 정민재
4.소속기관: 漢城大學校 大學院


1.번호: 96
2.논문제목: 가금류 악성 전염병 감염축 처리자의 보호 장비 착용실태 조사 및 개선에 관한 연구
3.저자: 임의형
4.소속기관: 전북대학교 일반대학원


1.번호: 97
2.논문제목: 環境影響을 低減하기 위한 傳染病 豫防法의 改善方向에 관한 연구
3.저자: 허인교
4.소속기관: 全北大學校


1.번호: 98
2.논문제목: Stability of an SIRS epidemic model with time delay
3.저자: 채경숙
4.소속기관: 부산대학교


1.번호: 99
2.논문제목: Bacteriophage application against reptile-associated bacterial zoonoses
3.저자: 권준
4.소속기관: 서울대학교 대학원


1.번호: 100
2.논문제목: Timeliness and trade-off between timeliness and PPV of the Korean national notifiable disease surveillance system
3.저자: 유효순
4.소속기관: 서울대학교 보건대학원


1.번호: 101
2.논문제목: Enhanced Resistance against Soil-borne Diseases by Root-Specific Expression of J1-1 in Transgenic Tobacco Plant
3.저자: 김진솔
4.소